In [1]:
import keras
from keras import layers
from keras.datasets import mnist
from keras.losses import BinaryCrossentropy
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
one_train = []
one_test = []
other_test = []
for i in range(x_train.shape[0]):
    if y_train[i] == 1:
        one_train.append(x_train[i])
        
for i in range(x_test.shape[0]):
    if y_test[i] == 1:
        one_test.append(x_test[i])
    else:
        other_test.append(x_test[i])

one_train = np.array(one_train)
one_test = np.array(one_test)
other_test = np.array(other_test)

'選擇是否加入噪音進行訓練'
'one_train加入noise'
x_noise = np.array(one_train)
x_noise = x_noise + 0.2 * np.random.normal(0, 1, size=x_noise.shape)
x_noise = np.clip(x_noise, 0., 1.)
one_train = np.concatenate([one_train, x_noise])

'one_test加入noise'
x2_noise = np.array(one_test)
x2_noise = x2_noise + 0.2 * np.random.normal(0, 1, size=x2_noise.shape)
x2_noise = np.clip(x2_noise, 0., 1.)
one_test = np.concatenate([one_test, x2_noise])

'other_test加入noise'
y_noise = np.array(other_test)
y_noise = y_noise + 0.2 * np.random.normal(0, 1, size=y_noise.shape)
y_noise = np.clip(y_noise, 0., 1.)
other_test = np.concatenate([other_test, y_noise])

In [ ]:
input_img = keras.Input(shape=(784,))
encoded = layers.Dense(128, activation='relu')(input_img)
encoded = layers.Dense(64, activation='relu')(encoded)
encoded = layers.Dense(32, activation='relu')(encoded)

decoded = layers.Dense(64, activation='relu')(encoded)
decoded = layers.Dense(128, activation='relu')(decoded)
decoded = layers.Dense(784, activation='sigmoid')(decoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(one_train, one_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(one_test, one_test))

Train on 6742 samples, validate on 1135 samples
Epoch 1/50


In [ ]:
one_de = autoencoder.predict(one_test)
other_de = autoencoder.predict(other_test)

one_loss_lst = []
other_loss_lst = []
 
def cross_entropy(targets, predictions):
    N = len(predictions)
    ce = -np.sum(targets * np.log(predictions)) / N
    return ce

for i in range(100):
    loss = cross_entropy(one_test[i], one_de[i])
    one_loss_lst.append(loss)

plt.hist(one_loss_lst)
plt.xlim(-.1, .5)
plt.show()

for i in range(100):
    loss = cross_entropy(other_test[i], other_de[i])
    other_loss_lst.append(loss)

plt.hist(other_loss_lst)
plt.xlim(-.1, .5)
plt.show()